# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
# !ls 

In [2]:
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [3]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

# reshape to 2d feature
train = train.reshape(-1, 1, 11, 39)
test = test.reshape(-1, 1, 11, 39)

Loading data ...


In [5]:
print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Size of training data: (1229932, 1, 11, 39)
Size of testing data: (451552, 1, 11, 39)


## Create Dataset

In [6]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [7]:
# VAL_RATIO = 0.2
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 1, 11, 39)
Size of validation set: (245987, 1, 11, 39)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [8]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [9]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

232

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [19]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 256, (5, 1))
        # self.pool1 = nn.MaxPool2d((1, 2))
        self.bn1 =  nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 128, (5, 1))
        # self.pool2 = nn.MaxPool2d((1, 2))
        self.bn2 =  nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 64, (1, 33))
        # self.pool3 = nn.MaxPool2d((1, 2))
        self.bn3 =  nn.BatchNorm2d(64)
        # self.conv4 = nn.Conv2d(64, 64, (3, 7))
        # self.bn4 =  nn.BatchNorm2d(64)
        # self.conv5 = nn.Conv2d(64, 32, (3, 7))
        # self.bn5 =  nn.BatchNorm2d(32)

        self.flat = nn.Flatten()

        self.linear1 = nn.Linear(1344, 512) 
        self.bn_l1 =  nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 128) 
        self.bn_l2 =  nn.BatchNorm1d(128)
        self.out = nn.Linear(128, 39) 
        self.relu = nn.ReLU()
        # self.relu = nn.LeakyReLU(0.1)
        self.act_fn = nn.Sigmoid()

        self.drop1 = nn.Dropout(0.2)
        self.drop2 = nn.Dropout(0.2)
        self.drop3 = nn.Dropout(0.2)

    def forward(self, x):
        x = self.conv1(x)
        # x = self.drop1(x)
        # x = self.pool1(x)
        x = self.relu(x)
        x = self.bn1(x)
    
        x = self.conv2(x)
        # x = self.pool2(x)
        x = self.relu(x)
        x = self.bn2(x)
        
        x = self.conv3(x)
        # x = self.pool3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.drop2(x)

        # x = self.conv4(x)
        # x = self.bn4(x)
        # x = self.relu(x)

        # x = self.conv5(x)
        # x = self.bn5(x)
        # x = self.relu(x)

        x = self.flat(x)

        x = self.linear1(x)
        # x = self.drop3(x)
        x = self.relu(x)
        x = self.bn_l1(x)

        x = self.linear2(x)
        x = self.relu(x)
        x = self.out(x)
        
        return x

## Training

In [11]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [12]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [21]:
# fix random seed for reproducibility
same_seeds(52728)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 30               # number of training epoch
learning_rate = 1e-4       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
# model.load_state_dict(torch.load(model_path))
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

DEVICE: cuda


In [22]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


  0%|          | 4/15375 [00:00<07:15, 35.31it/s][001/030] Train Acc: 0.650660 Loss: 1.120724 | Val Acc: 0.703387 loss: 0.915510
saving model with acc 0.703
  0%|          | 4/15375 [00:00<07:08, 35.85it/s][002/030] Train Acc: 0.715864 Loss: 0.874174 | Val Acc: 0.719136 loss: 0.853042
saving model with acc 0.719
  0%|          | 4/15375 [00:00<06:46, 37.82it/s][003/030] Train Acc: 0.741643 Loss: 0.783459 | Val Acc: 0.721420 loss: 0.850273
saving model with acc 0.721
  0%|          | 3/15375 [00:00<09:28, 27.04it/s][004/030] Train Acc: 0.760684 Loss: 0.719842 | Val Acc: 0.721489 loss: 0.854977
saving model with acc 0.721
  0%|          | 7/15375 [00:00<03:47, 67.56it/s][005/030] Train Acc: 0.775922 Loss: 0.669599 | Val Acc: 0.722928 loss: 0.853893
saving model with acc 0.723
  0%|          | 5/15375 [00:00<05:25, 47.17it/s][006/030] Train Acc: 0.788315 Loss: 0.629041 | Val Acc: 0.722998 loss: 0.870527
saving model with acc 0.723
100%|██████████| 15375/15375 [04:14<00:00, 60.47it/s]
[030

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [26]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [27]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [28]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))